[Keras-Tuner](https://github.com/keras-team/keras-tuner) is a convenient solution for hyperparameter tuning. It provides many built-in utilities with latest Keras APIs. 

In this notebook I will show using a HyperModel available in Keras-Tuner that utilizes [Keras preprocessing layers (KPL)](https://keras.io/guides/preprocessing_layers/) for a hyperparameter tunable image augmentation, combined with a hypermodel for EfficientNet based on newly added implementation in [keras.applications](https://keras.io/api/applications/). Simply applying default setting yields good result.

This notebook requires TF>=2.3.0, and Keras-Tuner installed from [github head](https://github.com/keras-team/keras-tuner@master).

In general, hyperparameter searching requires large amount of resources. In the example you can choose `TESTING_LEVEL = 0` for a quick debug, `TESTING_LEVEL = 1` for a relatively short test to see some reasonable result (~0.9), or `TESTING_LEVEL = 2` and set your own search epochs limit for an extended run.

*Note* Keras preprocessing layers is in experimental stage as of TF2.3. It is not fully supporting TPUs, so this notebook will only run on GPUs depite working on this TPU competition. A TPU enabled version that do not use KPL is [here](https://www.kaggle.com/fuyixing/flower-classification-on-tpu-with-keras-tuner).

In [ ]:
# Configuration
TESTING_LEVEL = 0

if TESTING_LEVEL == 0:
    # For debugging
    IMAGE_SIZE = [224, 224]
    EPOCHS_SEARCH = 5
    MAX_TRIALS = 3
    EPOCHS_FINAL = 50
    BATCH_SIZE_PER_REPLICA = 16
elif TESTING_LEVEL == 1:
    # For relatively short test to see some reasonable result
    IMAGE_SIZE = [224, 224]
    EPOCHS_SEARCH = 10
    MAX_TRIALS = 5
    EPOCHS_FINAL = 10
    BATCH_SIZE_PER_REPLICA = 16
else:
    # For an extended run.
    # Can set even larger MAX_TRIALS and EPOCHS_SEARCH for even better result.
    IMAGE_SIZE = [224, 224]
    EPOCHS_SEARCH = 30
    MAX_TRIALS = 10
    EPOCHS_FINAL = 10
    BATCH_SIZE_PER_REPLICA = 32
    

# load previous keras tuner result from output if possible
!cp -r ../input/flower-kt-record/flower_classification_kt_kpl .

In [ ]:
!pip install -q tensorflow==2.3.0
!pip install -q git+https://github.com/keras-team/keras-tuner@master

In [ ]:
import random, re, math
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf, tensorflow.keras.backend as K
from kaggle_datasets import KaggleDatasets
print('Tensorflow version ' + tf.__version__)
import kerastuner as kt
tf.config.optimizer.set_jit(True)

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # Sync TPU version
    from cloud_tpu_client import Client
    c = Client()
    c.configure_tpu_version(tf.__version__, restart_type='ifNeeded')
    
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None
    

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

Here we set a few over-all hyperparameters. These may also be searched through Keras-Tuner with customized tuner classes and model bulding function / HyperModels. 

# Data preparation
The code for data preparation are mostly adapted from this [notebook](https://www.kaggle.com/cdeotte/rotation-augmentation-gpu-tpu-0-96) with some minor modification. Note that I am converting labels to one-hot encoding.

In [ ]:
# Data access
GCS_DS_PATH = KaggleDatasets().get_gcs_path('tpu-getting-started')
print(GCS_DS_PATH)

GCS_PATH_SELECT = { # available image sizes
    192: GCS_DS_PATH + '/tfrecords-jpeg-192x192',
    224: GCS_DS_PATH + '/tfrecords-jpeg-224x224',
    331: GCS_DS_PATH + '/tfrecords-jpeg-331x331',
    512: GCS_DS_PATH + '/tfrecords-jpeg-512x512'
}

GCS_PATH = GCS_PATH_SELECT[IMAGE_SIZE[0]]

TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec') # predictions on this dataset should be submitted for the competition

In [ ]:
CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']                                                                                                                                               # 100 - 102

## Dataset functions

Adapted from starter [kernel][1]. Notice that augmentation is not included as a part of data pipeline. Instead, it will be in the model.

[1]: https://www.kaggle.com/mgornergoogle/getting-started-with-100-flowers-on-tpu

In [ ]:
from tensorflow.data.experimental import AUTOTUNE

def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) 
    # For keras.application.efficientnet, inputs range [0, 255]
    image = tf.ensure_shape(image, [*IMAGE_SIZE, 3])
    return image

def read_labeled_tfrecord(example, num_classes=len(CLASSES)):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "class": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    label = tf.one_hot(label, num_classes)
    return image, label

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "id": tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords.
    # When ordering is not needed, set `ordered=False` for faster loading.
    
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False 

    # automatically interleaves reads from multiple files
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE)
    # use data as soon as it streams in, rather than in its original order
    dataset = dataset.with_options(ignore_order)
    if labeled:
        dataset = dataset.map(read_labeled_tfrecord,
                              num_parallel_calls=AUTOTUNE)
    else:
        dataset = dataset.map(read_unlabeled_tfrecord,
                              num_parallel_calls=AUTOTUNE)
    return dataset


def get_training_dataset(filenames):
    dataset = load_dataset(filenames, labeled=True, ordered=False)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    # Drop remainder to ensure same batch size for all.
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    # Prefetch next batch while training
    dataset = dataset.prefetch(AUTOTUNE) 
    return dataset

def get_validation_dataset(filenames):
    dataset = load_dataset(filenames, labeled=True, ordered=False)
    # Drop remainder to ensure same batch size for all.
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    # Prefer not prefetching for validation data on GPUs.
    # dataset = dataset.prefetch(AUTOTUNE)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=False)
    return dataset

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [ ]:
num_val_samples = count_data_items(VALIDATION_FILENAMES)
num_train_samples = count_data_items(TRAINING_FILENAMES)

num_train_batches = num_train_samples // BATCH_SIZE
num_val_batches = num_val_samples // BATCH_SIZE

train_ds = get_training_dataset(TRAINING_FILENAMES)
validation_ds = get_validation_dataset(VALIDATION_FILENAMES)
all_ds = get_training_dataset(TRAINING_FILENAMES + VALIDATION_FILENAMES)

# Search Hyper-parameters with Keras-Tuner

Now we will search hyperparameters with Keras-Tuner.

A HyperModel in Keras-Tuner is class with a `build` method that creates a *compiled* Keras model using a set of hyperparameters for each trial. A tuner takes a [HyperModel](https://keras-team.github.io/keras-tuner/documentation/hypermodels/) or simply a model builder function, and tries the combinations of the hyperparameters for times depending on different tuning algorithms (defined by [Oracle](https://keras-team.github.io/keras-tuner/documentation/oracles/)). Each of the built-in [Tuner](https://keras-team.github.io/keras-tuner/documentation/tuners/) have corresponding oracle.

For simplicity, in this example I only using pre-built HyperModels and Tuner. It is also possible to create any HyperModel or model-building function, to create custom tuning algorithms by subclassing Oracles, and to use custom training loop by [subclassing Tuner](https://keras-team.github.io/keras-tuner/tutorials/subclass-tuner/). Customized training loop can help fully utilizing the pretrained weights by first training some epoches with pretrained layers frozen. 



## Data Augmentation with Keras-Tuner HyperModel

Keras-Tuner provides a built-in image augmenting hypermodel in applications. This HyperModel, `HyperImageAugment`, can either be used as a stand-alone hypermodel or as a part of other HyperModels. It is especially convenient to be used with HyperModel in applications shipped with Keras-Tuner, as this notebook will later demonstrate. Here we first build some sample HyperModel for augmentation to visualize the augmentation.

In [ ]:
# function for visualizing augmentation model
def visualize(aug_model):
    row = 3; col = 4;
    element = train_ds.unbatch().take(1)

    for (img, label) in element:
        img_batch = tf.repeat(tf.expand_dims(img, 0),row * col,axis=0)
        img_augment = aug_model(img_batch)
        plt.figure(figsize=(15, int(15 * row / col)))
        for j in range(row * col):
            plt.subplot(row,col, j + 1)
            plt.axis('off')
            plt.imshow(img_augment[j,] / 255.)
        plt.show()
        break

`HyperImageAugment` implements two types of image augmentations, controlled by choosing `augment_layers` to be 0 or a positive integer. When `augment_layers` is zero, it sequentially apply selected preprocessing layers; otherwise it implements a [RandAugment](https://arxiv.org/abs/1909.13719) style augmentation using the selected layers. 

For demonstration purpose, we first build a hypermodel for augmentation that sequentially apply all layers, selecting only rotation and vertical translation. We set the factor for `rotate` and `translate_y` to be both 0.3 to 0.5, so for each trial, the augmentation factor for each layer is selected in the ranges defined in hypermodel. The augmentation factors for KPLs are mostly designed to be between 0 and 1, whereas each example are augmented with a random magnitude between 0 and the factor.

In [ ]:
from kerastuner.applications import HyperImageAugment

hm_aug = HyperImageAugment(
    input_shape=[*IMAGE_SIZE, 3],
    augment_layers=0,
    rotate=[0.3, 0.5], # range of factor of rotation
    translate_x=None, # horizontal translation is off
    translate_y=[0.3, 0.5], # range of factor of vertical translation
    contrast=None) # auto contrast is off

This can both be seen directly from model summary, or through example data. Notice that all images are both translating and rotating. Also, the translation / rotation are as low as 0 because we are only setting the minimum of `factor` to be 0.3, not minimum of actual per-sample transform to 0.3. 

In [ ]:
from kerastuner.engine.hyperparameters import HyperParameters
hp = HyperParameters()
aug_model = hm_aug.build(hp)
aug_model.summary()

visualize(aug_model)

Now we build a hypermodel for RandAugment selecting rotation and vertical translation. We set the factor for `rotate` and `translate_y` to be both 0.3 to 0.5. For each trial, the augmentation factor for each layer is selected in the ranges defined in hypermodel. We set `augment_layers=[1, 1]` so that there is always one layer applied, either `rotate` or `translate_y` to each *sample*. Hence each figure is either rotated or translated (but not both in this case).

In [ ]:
from kerastuner.applications import HyperImageAugment

hm_aug = HyperImageAugment(
    input_shape=[*IMAGE_SIZE, 3],
    augment_layers=[1, 1], # only one layer of augmentation
    rotate=[0.3, 0.5], # range of factor of rotation
    translate_x=None, # horizontal translation is off
    translate_y=[0.3, 0.5], # range of factor of vertical translation
    contrast=None) # auto contrast is off

hp = HyperParameters()
aug_model = hm_aug.build(hp)
aug_model.summary()

visualize(aug_model)

For our searching and training, I will simply use the default options for HyperImageAugment. That is, up to 3 layers of RandAugment style augment with horizontal / vertial translation factor searched in range $[0, 0.4]$, rotation with factor in $[0, 0.5]$, random contrast with factor in $[0, 0.3]$.

In [ ]:
from kerastuner.applications import HyperImageAugment
# Use default setting
hm_aug = HyperImageAugment(input_shape=[*IMAGE_SIZE, 3]) 

## The EfficientNet HyperModel

Keras Tuner provides hypermodel for EfficientNet in applications. It utilizes EfficientNet from `tf.keras.applications`, and by default uses weights pretrained on ImageNet. Currently this hypermodel searches version of EfficientNet (B0-B7), type of pooling layer after feature extration (max pooling or average pooling) and initial learning rate. 

*A side note*: TF2.3 provides `experimental_steps_per_execution` keyword for `model.compile`. This greatly improves TPU efficiency, and may also slightly improve GPU efficiency.

In [ ]:
# Define HyperModel using built-in application
from kerastuner.applications.efficientnet import HyperEfficientNet

hm = HyperEfficientNet(
    input_shape=[*IMAGE_SIZE, 3],
    classes=len(CLASSES),
    # Augmentation model goes here. It can be HyperModel or Keras Model.
    augmentation_model=hm_aug) 

# Optional: Restrict default hyperparameters.
# To take effect, pass this `hp` instance when constructing tuner as `hyperparameters=hp`
from kerastuner.engine.hyperparameters import HyperParameters
hp = HyperParameters()
# Restrict choice of EfficientNet version from B0-B7 to B0-B3
hp.Choice('version', ['B0', 'B1', 'B2', 'B3']) 


To fully take advantage of pretrained weights, it is best to first freeze the feature extracting stage of the model; and fine tune the entire model for a few epochs. Here we show this work flow by subclassing the tuner. For a relatively easy task like this one, training EfficientNet from scratch is already capable of reaching good result; but utilizing pretrained weight can siginficantly accelerate convergence.

In [ ]:
import copy

# Helper function: re-compile with the same loss/metric/optimizer
def recompile(model):
    metrics = model.compiled_metrics.metrics
    metrics = [x.name for x in metrics]
    model.compile(loss=model.loss,
                  metrics=metrics,
                  optimizer=model.optimizer)

class FineTuner(kt.engine.tuner.Tuner):
    def run_trial(self, trial, *fit_args, **fit_kwargs):       
        copied_fit_kwargs = copy.copy(fit_kwargs)
        callbacks = fit_kwargs.pop('callbacks', [])
        callbacks = self._deepcopy_callbacks(callbacks)
        copied_fit_kwargs['callbacks'] = callbacks
        
        tf.keras.backend.clear_session()
        model = self.hypermodel.build(trial.hyperparameters)
        #dry run to build metrics
        model.evaluate(*fit_args, steps=1, batch_size=1)
        
        # freeze pretrained feature extractor
        for l in model.layers:
            # For efficientnet implementation we use, layers in the
            # Feature extraction part of model all have 'block', 
            # 'stem' or 'top_conv' in name.
            if any(x in l.name for x in ['block', 'stem', 'top_conv']):
                l.trainable = False
            if isinstance(l, tf.keras.layers.BatchNormalization):
                l.trainable = True
        # it usually suggested to increase learning rate if running with
        # multiple replica because of the increased batch size.
        model.optimizer.lr = model.optimizer.lr * strategy.num_replicas_in_sync
        recompile(model)
        model.fit(*fit_args, **copied_fit_kwargs)
        
        for l in model.layers:
            l.trainable = True
        model.optimizer.lr = model.optimizer.lr / 10
        recompile(model)
        
        # TunerCallback reports results to the `Oracle` and save the trained Model.
        callbacks.append(kt.engine.tuner_utils.TunerCallback(self, trial))
        
        model.fit(*fit_args, **copied_fit_kwargs)

When `Tuner` is customized, we need to initiate `Oracle` and pass the `Oracle` object to initiate the `Tuner` object. If using built-in tuners, we can directly initiate tuners such as `RandomSearch`. 

In [ ]:
# Define Oracle
oracle = kt.tuners.bayesian.BayesianOptimizationOracle(
    objective='val_accuracy',
    max_trials=MAX_TRIALS,
    hyperparameters=hp,
)

# Initiate Tuner
tuner = FineTuner(
    hypermodel=hm,
    oracle=oracle,
    directory='flower_classification_kt_kpl',
    project_name='bayesian_efficientnet',
    # Distribution strategy is passed in here.
    distribution_strategy=strategy, 
    # optimizer can be set here.
    optimizer='adam',
    metrics=['accuracy'],
    )
tuner.search_space_summary()

`tuner.search()` is called just like the way you would call `model.fit()`.

In [ ]:
# tuner.search(train_ds,
#              epochs=EPOCHS_SEARCH,
#              validation_data=validation_ds,
#              steps_per_epoch=num_train_batches,
#              callbacks=[tf.keras.callbacks.ReduceLROnPlateau(),
#                         tf.keras.callbacks.EarlyStopping(patience=5)],
#              verbose=2)

As long as some trials are complete, we may move on to get the best result up to now even if search fail to finish. Also, as long as the project directory is not deleted, you may run the same code and it will pick up from where it stopped.

In [ ]:
tuner.results_summary()
model = tuner.get_best_models()[0]

It is usually good to fit the best model with all data including validation data after hyperparameter search is done.

In [ ]:
# Train the best model with all data
model.fit(all_ds,
          epochs=EPOCHS_FINAL,
          steps_per_epoch=num_train_batches + num_val_batches,
          callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='loss')],
          verbose=2)

# Prediction and create submission file

In [ ]:
ds_test = get_test_dataset(ordered=True)

print('Computing predictions...')
predictions = []

for i, (test_img, test_id) in enumerate(ds_test):
    print('Processing batch ', i)
    probabilities = model(test_img)
    prediction = np.argmax(probabilities, axis=-1)
    predictions.append(prediction)

predictions = np.concatenate(predictions)
print('Number of test examples predicted: ', predictions.shape)

In [ ]:
# Get image ids from test set and convert to unicode
ds_test_ids = ds_test.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(ds_test_ids.batch(np.iinfo(np.int64).max))).numpy().astype('U')

# Write the submission file
np.savetxt(
    'submission.csv',
    np.rec.fromarrays([test_ids, predictions]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='id,label',
    comments='',
)

# Look at the first few predictions
!head submission.csv